In [1]:
import caffe
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
import caffe.draw
from caffe.proto import caffe_pb2
from google.protobuf import text_format
import cv2

In [2]:
import sys
reload(sys)
sys.setdefaultencoding('utf-8')

In [3]:
caffe.set_mode_cpu()

Deploy network

In [39]:
net = caffe_pb2.NetParameter()
text_format.Merge(open('mnist_autoencoder_deploy.prototxt').read(), net)
image_name = 'mnist_autoencoder_deploy_modified.png'
print('Drawing net to %s' % image_name)

phase = caffe.TRAIN
caffe.draw.draw_net_to_file(net, image_name, 'LR', phase)

In [40]:
net = caffe.Net('mnist_autoencoder_deploy.prototxt', '_iter_60000.caffemodel', caffe.TEST)

In [41]:
shapes = [(k, v.data.shape) for k, v in net.blobs.items()]
for shape in shapes:
    print shape

In [48]:
input_directory = u'C:\\Users\\Mikhail\\Diploma\\Mnist_prepare\\Black_background\\test\\1\\'

import scipy.misc

iteration = 0

for filename in os.listdir(input_directory):
    image = Image.open(input_directory + filename)

    X = np.zeros((1, 1, 28, 28), dtype='f4') 
    X[0] = image
    for i in range(X.shape[2]):
        for j in range(X.shape[3]):
            X[0][0][i][j] = X[0][0][i][j] * 0.0039215684
            
    input_vector = X[0].flatten()
    
    net.blobs['Input'].data[...] = X[0]
    
    net.forward()
    
    result_vector = np.array(net.blobs['decode1neuron'].data, dtype='f4')
    
    result_vector = result_vector.flatten()
    
    print np.linalg.norm(input_vector - result_vector)
    
    for i in range(len(result_vector)):
        result_vector[i] = result_vector[i] * 255
    
    result_matrix = np.reshape(result_vector, (28, 28))
    scipy.misc.imsave('1//' + filename, result_matrix)
    
    iteration += 1
    if iteration == 20:
        break